## 1. 네이버API 사용 크롤링

In [1]:
import time
import os
import sys
import urllib.request
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
curl "https://openapi.naver.com/v1/papago/n2mt" \
-H "Content-Type: application/x-www-form-urlencoded; charset=UTF-8" \
-H "X-Naver-Client-Id: vsdXXZ9ntwAeFeejf4qC" \
-H "X-Naver-Client-Secret: zHa2LO_TlI" \
-d "source=ko&target=en&text=만나서 반갑습니다." -v

In [ ]:
client_id = "YOUR_CLIENT_ID"
client_secret = "YOUR_CLIENT_SECRET"

In [ ]:
# 네이버 검색 API예제는 블로그를 비롯 전문자료까지 호출방법이 동일하므로 blog검색만 대표로 예제를 올렸습니다.
# 네이버 검색 Open API 예제 - 블로그 검색

client_id = "YOUR_CLIENT_ID"
client_secret = "YOUR_CLIENT_SECRET"
encText = urllib.parse.quote("검색할 단어")
url = "https://openapi.naver.com/v1/search/blog?query=" + encText # json 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

#### Dictionary 형태로 변환

In [ ]:
json_text = eval(response_body)
json_text

In [ ]:
type(json_text)

In [ ]:
json_text['display']

In [ ]:
json_text['items'][0]['description']

In [ ]:
json_text['items'][0]['link']

In [ ]:
print(json_text['items'][0]['link'])

#### pandas 데이터프레임 변환

In [ ]:
news_df = pd.DataFrame(json_text)

In [ ]:
news_df

In [ ]:
news_df['items'][0]

In [ ]:
print(news_df['items'][0]['link'])

In [ ]:
news_df['items'][0]['description']

### 네이버API - 뉴스

In [ ]:
#네이버 검색 Open API 예제 - 뉴스 검색

client_id = "YOUR_CLIENT_ID"
client_secret = "YOUR_CLIENT_SECRET"
encText = urllib.parse.quote("트와이스")
url = "https://openapi.naver.com/v1/search/news?query=" + encText # json 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

In [ ]:
type(response_body)

In [ ]:
url_enter = 'https://entertain.naver.com/read?oid=417&aid=0000458809'
print(url_enter)

In [ ]:
import requests

html = requests.get(url_enter)
print(html)

In [ ]:
html.text

In [ ]:
soup1 = BeautifulSoup(html.text, 'lxml')
print(soup1)

In [ ]:
# title 정보에 접근
soup1.title.text

In [ ]:
from urllib.request import urlopen

html = urlopen(url_enter)
print(html)

In [ ]:
from bs4 import BeautifulSoup

soup2 = BeautifulSoup(html, 'lxml')
print(soup2)

In [ ]:
# title 정보에 접근
soup2.title.text

In [ ]:
soupx = soup2

media = soupx.find('meta',attrs={'name':'twitter:creator'}).get('content').strip() # media
title = soupx.find('meta',attrs={'property':'og:title'}).get('content').strip() # title
url = soupx.find('meta', attrs={'property':'og:url'}).get('content') # URL
section = soupx.find('meta', attrs={'name':'twitter:site'}).get('content') # Section
gdid = url.split('aid=')[1].split("&")[0]
body  = soupx.find('div', attrs={'class':'article_body'}).text.strip() # body

try: category = soupx.find('meta', attrs={'property':'me2:category2'}).get('content') # category
except: category = ''

news_info={'0gdid':gdid, '1title':title, '2url':url, '3section':section, '4category':category, '5body':body}
print(news_info)

In [ ]:
news_info['5body']

### 반복 수행을 위한 함수 작성 예시

In [ ]:
def get_article(url):
    html = urlopen(url)
    time.sleep(1) # 반드시 time.sleep() 시간차 함수를 넣기 바람
    soup = BeautifulSoup(html, 'lxml')
    #print(url)
    return soup

In [ ]:
# '네이버'뉴스에 게재된 '연예'뉴스 포맷에만 적용 가능
def extract_information_enternews(soupx):
    media = soupx.find('meta',attrs={'name':'twitter:creator'}).get('content').strip() # media
    title = soupx.find('meta',attrs={'property':'og:title'}).get('content').strip() # title
    url = soupx.find('meta', attrs={'property':'og:url'}).get('content') # URL
    section = soupx.find('meta', attrs={'name':'twitter:site'}).get('content') # Section
    gdid = url.split('aid=')[1].split("&")[0]
    body  = soupx.find('div', attrs={'class':'article_body'}).text.strip() # body

    try: category = soupx.find('meta', attrs={'property':'me2:category2'}).get('content') # category
    except: category = ''
            
    news_info={'0gdid':gdid, '1title':title, '2url':url, '3section':section, '4category':category, '5body':body}

    return news_info

#### 함수를 사용해 웹페이지 소스에서 정보 추출

In [ ]:
url_enter

In [ ]:
soupx_enter = soup2
extract_information_enternews(soupx_enter)

##### 네이버 뉴스가 아닌 경우

In [ ]:
url_not_naver = json_text['items'][2]['link']
print(json_text['items'][2])
print(url_not_naver)

In [ ]:
soupx_not_naver = get_article(url_not_naver)
extract_information_enternews(soupx_not_naver) # 에러 발생

#### 네이버 뉴스 중, '연예' 뉴스가 아닌 경우

In [ ]:
url_social='https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&oid=025&aid=0002944990&sid1=001'
print(url_social)

In [ ]:
soupx_social = get_article(url_social)
extract_information_enternews(soupx_social)

### 반복문을 이용한 정보 추출

In [ ]:
for i in range(10):
    url_enter = json_text['items'][i]['link']
    print(url_enter) # 네이버 연예뉴스가 아닐 경우, 에러 및 중단 발생
    soupx_enter = get_article(url_enter)
    extract_information_enternews(soupx_enter)

In [ ]:
# try-except 구문 활용
for i in range(10):
    url_enter = json_text['items'][i]['link']
    print(url_enter)
    try: # 에러가 발생하지 않을 경우, 다음의 명령문 실행
        soupx_enter = get_article(url_enter)
        print(extract_information_enternews(soupx_enter)) # 추출한 정보 출력
    except:
        pass  # 에러가 발생할 경우, (중단하지 않고) 다음 i 차례로 넘어감

##### 예시) 1년치 뉴스를 수집하는 경우

In [ ]:
url_str_day1 = [x for x in ['01.01','01.16','02.01']]
url_str_day2 = [x for x in ['01.15','01.31','02.15']]
#url_str_day1 = [x for x in ['01.01','01.16','02.01','02.16','03.01','03.16','04.01','04.16','05.01','05.16','06.01','06.16',
#                            '07.01','07.16','08.01','08.16','09.01','09.16','10.01','10.16','11.01','11.16','12.01','12.16']]
#url_str_day2 = [x for x in ['01.15','01.31','02.15','02.28','03.15','03.31','04.15','04.30','05.15','05.31','06.15','06.30',
#                            '07.15','07.31','08.15','08.31','09.15','09.30','10.15','10.31','11.15','11.30','12.15','12.31']]
​
naver_search_url = []
search_result_count_sum = 0
for k in range(len(url_str_day1)):
    url_search_str0 = 'https://search.naver.com/search.naver?&where=news&query=%22%5B%EB%8B%A8%EB%8F%85%5D%22&sm=tab_pge&sort=2&photo=0&field=1&reporter_article=&pd=3&ds=2018.'
    url_search_str_add1 = url_str_day1[k] #'01.01'
    url_search_str_add2 = '&de=2018.' + url_str_day2[k] #'12.31'
    url_search_str_add3 = '&docid=&nso=so:da,p:from2018' + url_str_day1[k].replace('.','') #'0101'
    url_search_str_add4 = 'to2018' + url_str_day2[k].replace('.','') #'1231'
    url_search_str_add5 = ',a:t&mynews=101&start='
    url_search_str0 = url_search_str0+url_search_str_add1+url_search_str_add2+url_search_str_add3+url_search_str_add4+url_search_str_add5
    url_search_firstpage = url_search_str0 + '1&refresh_start=0'
    naver_search_url.append(url_search_firstpage)
    soup_naver_search_firstpage = get_article(url_search_firstpage)
    search_result_pages = soup_naver_search_firstpage.find('div', attrs={'class':'title_desc all_my'}).text.split()[2]
    search_result_count = int(search_result_pages.replace('건','').replace(',',''))
    search_result_count_sum += search_result_count
    search_result_count_pages = int(search_result_count/10)
    print(url_search_firstpage, search_result_pages, search_result_count_pages)
    #for j in range(0, search_result_count_pages):
    for j in range(0, 1):
        jx = j*10 + 11
        url_search_pages = url_search_str0 + str(jx) + '&refresh_start=0'
        naver_search_url.append(url_search_pages)
        #print(url_search_pages)
print(search_result_count_sum)